In [1]:
import numpy as np
import requests

In [2]:
WEB_API = r"https://networks.skewed.de/api"

# Basic list of network names
response = requests.get(f"{WEB_API}/nets")
network_names = response.json()  # This gives a list of names

# Filtered by tags
response = requests.get(f"{WEB_API}/nets?full=True")
networks_dict = response.json()


In [3]:
for name, data in networks_dict.items():
    if not ("Social" in data["tags"] and "Offline" in data["tags"]):
        continue
    
    try: # single data set
        num_nodes = data["analyses"]["num_vertices"]
        
        if not (num_nodes > 100 and num_nodes < 5000000):
            continue

    except KeyError: #multiple data sets
        num_nodes_list = [ds["num_vertices"] for ds in list(data["analyses"].values())]
        num_nodes = [n for n in num_nodes_list if n > 100 and n < 5000000]
        
    print(name, "\t  Nodes: ", num_nodes, data["tags"]) 
    print("  ", data["description"])
    print("   ")



add_health 	  Nodes:  [685, 426, 586, 689, 569, 1089, 795, 1246, 290, 495, 109, 922, 380, 626, 679, 876, 828, 563, 1229, 1168, 578, 746, 900, 877, 2250, 1681, 873, 2209, 427, 542, 1008, 291, 2037, 2100, 1512, 1723, 1147, 1047, 1550, 998, 1184, 1929, 159, 2587, 790, 1799, 647, 1107, 357, 467, 1282, 1778, 1007, 111, 1216, 1898, 1111, 710, 566, 655, 530, 1460, 904, 457, 358, 1650, 730, 1011, 436, 1235, 205, 624, 1383, 946, 1338, 1755, 254] ['Social', 'Offline', 'Weighted']
   A directed network of friendships obtained through a social survey of high school students in 1994. The ADD HEALTH data are constructed from the in-school questionnaire; 90,118 students representing 84 communities took this survey in 1994-95. Some communities had only one school; others had two. Where there are two schools in a community students from one school were allowed to name friends in the other, the "sister school".[^icon]

Each student was given a paper-and-pencil questionnaire and a copy of a roster listin

In [4]:
for name, data in networks_dict.items():
    if not ("Social" in data["tags"] and "Online" in data["tags"]):
        continue
    
    try: # single data set
        num_nodes = data["analyses"]["num_vertices"]
        
        if not (num_nodes > 100 and num_nodes < 5000000):
            continue

    except KeyError: #multiple data sets
        num_nodes_list = [ds["num_vertices"] for ds in list(data["analyses"].values())]
        num_nodes = [n for n in num_nodes_list if n > 100 and n < 5000000]
        
    print(name, "\t  Nodes: ", num_nodes, data["tags"]) 
    print("  ", data["description"])
    print("   ")

academia_edu 	  Nodes:  200169 ['Social', 'Online', 'Unweighted']
   Snapshot of the follower relationships among users of academia.edu, a platform for academics to share research papers, scraped in 2011. Nodes are users and a directed edge (i,j) denotes that user i follows j.[^icon]
[^icon]: Description obtained from the [ICON](https://icon.colorado.edu) project.
   
advogato 	  Nodes:  6541 ['Social', 'Online', 'Weighted']
   A network of trust relationships among users on Advogato, an online community of open source software developers. Edge direction indicates that node i trusts node j, and edge weight denotes one of four increasing levels of declared trust from i to j: observer (0.4), apprentice (0.6), journeyer (0.8), and master (1.0).[^icon]
[^icon]: Description obtained from the [ICON](https://icon.colorado.edu) project.
   
anybeat 	  Nodes:  12645 ['Social', 'Online', 'Unweighted']
   A snapshot of the Anybeat online social network from 2013, before it was shut down. Nodes ar

In [5]:
for name, data in networks_dict.items():
    if not ("Social" in data["tags"] and "Online" not in data["tags"] and "Offline" not in data["tags"]):
        continue
    
    try: # single data set
        num_nodes = data["analyses"]["num_vertices"]
        
        if not (num_nodes > 100 and num_nodes < 5000000):
            continue

    except KeyError: #multiple data sets
        num_nodes_list = [ds["num_vertices"] for ds in list(data["analyses"].values())]
        num_nodes = [n for n in num_nodes_list if n > 100 and n < 5000000]
        
    print(name, "\t  Nodes: ", num_nodes, data["tags"]) 
    print("  ", data["description"])
    print("   ")

arxiv_authors 	  Nodes:  [133280, 108300, 26197, 89209, 68746] ['Social', 'Collaboration', 'Unweighted', 'Projection']
   Scientific collaborations between authors of papers submitted to arxiv.org, under 5 categories: gr-qc, astro-ph, cond-mat, hep-ph, and hep-th categories, spanning January 1993 to April 2003. If an author i co-authored a paper with author j, the graph contains a undirected edge from i to j. If the paper is co-authored by k authors this generates a completely connected (sub)graph on k nodes.[^icon]
[^icon]: Description obtained from the [ICON](https://icon.colorado.edu) project.
   
arxiv_collab 	  Nodes:  [16706, 16726, 31163, 40421, 8361] ['Social', 'Collaboration', 'Weighted', 'Projection']
   Collaboration graphs for scientists, extracted from the Los Alamos e-Print arXiv (physics), for 1995-1999 for three categories, and additionally for 1995-2003 and 1995-2005 for one category. For copyright reasons, the MEDLINE (biomedical research) and NCSTRL (computer science

In [6]:
names_off_sing = [
    "contact",
    "residence_hall",
    "sp_primary_school",
    "highschool",
    "dnc",
    "email_company",
    "email_enron",
    "email_eu",
    "uni_email"
]
names_off_mult = [
    "add_health",
    "sp_high_school",
    "spanish_highschools"
]

names_on_sing = [
    "facebook_friends",
    "facebook_wall",
    "google_plus",
    "twitter",
    "twitter_15m",
    "livejournal_aminer",
    "livejournal",
    "pokec"
]
names_on_mult = [
    "facebook_organizations",
    "ego_social"
]

In [7]:
for name in names_off_sing:
    print(f"{name} \t  Nodes: ", networks_dict[name]["analyses"]["num_vertices"])

for name in names_off_mult:
    num_nodes_list = [(ind,ds["num_vertices"]) for ind, ds in list(networks_dict[name]["analyses"].items())]
    num_nodes = [n for n in num_nodes_list if n[1] > 400 and n[1] < 5000000]
    for ind, number in num_nodes:
        print(f"{name}_{ind} \t  Nodes: ", number)


contact 	  Nodes:  274
residence_hall 	  Nodes:  217
sp_primary_school 	  Nodes:  242
highschool 	  Nodes:  70
dnc 	  Nodes:  2029
email_company 	  Nodes:  167
email_enron 	  Nodes:  36692
email_eu 	  Nodes:  265214
uni_email 	  Nodes:  1133
add_health_comm10 	  Nodes:  685
add_health_comm11 	  Nodes:  426
add_health_comm12 	  Nodes:  586
add_health_comm13 	  Nodes:  689
add_health_comm14 	  Nodes:  569
add_health_comm15 	  Nodes:  1089
add_health_comm16 	  Nodes:  795
add_health_comm17 	  Nodes:  1246
add_health_comm19 	  Nodes:  495
add_health_comm20 	  Nodes:  922
add_health_comm22 	  Nodes:  626
add_health_comm23 	  Nodes:  679
add_health_comm24 	  Nodes:  876
add_health_comm25 	  Nodes:  828
add_health_comm26 	  Nodes:  563
add_health_comm27 	  Nodes:  1229
add_health_comm28 	  Nodes:  1168
add_health_comm29 	  Nodes:  578
add_health_comm30 	  Nodes:  746
add_health_comm31 	  Nodes:  900
add_health_comm32 	  Nodes:  877
add_health_comm33 	  Nodes:  2250
add_health_comm34 	  Nodes:

In [8]:
data_dict_off = {
    'school_friendship': [
        "spanish_highschools__1",
        "spanish_highschools__2",
        "spanish_highschools__6",
        "add_helath__comm7",
        "add_health__comm11",
        "add_health__comm12",
        "add_health__comm29",
    ],
    'email': [
        "email_enron",
        "uni_email",
        "email_eu",
        "dnc"
    ]
}

## Downloading

In [ ]:
import os
import zstandard as zstd

def decompress_zst(input_path, output_path=None):
    if output_path is None:
        output_path = os.path.splitext(input_path)[0]  # removes .zst
    
    with open(input_path, 'rb') as compressed:
        dctx = zstd.ZstdDecompressor()
        with open(output_path, 'wb') as output:
            dctx.copy_stream(compressed, output)
    
    print(f"Decompressed to: {output_path}")
    return output_path

def download_graphs(network_names, save_dir="downloads"):
    os.makedirs(save_dir, exist_ok=True)
    
    base_url = "https://networks.skewed.de/net/{name}/files/{net}.gt.zst"
    file_paths = []
    for full_name in network_names:
        if "__" in full_name:
            name, net = full_name.split("__")
        else:
            name = net = full_name

        url = base_url.format(name=name, net=net)
        file_path = os.path.join(save_dir, f"{name}.gt.zst")
        
        try:
            print(f"Downloading {name}...")
            response = requests.get(url, stream=True)
            response.raise_for_status()
            
            with open(file_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
                    
            print(f"✔ Downloaded {name} to {file_path}")
        
        except requests.exceptions.RequestException as e:
            print(f"✘ Failed to download {name}: {e}")

        file_paths.append(file_path)

        # Decompress the downloaded file
        try:
            decompress_zst(file_path)
        except Exception as e:
            print(f"✘ Failed to decompress {file_path}: {e}")




target_dir = r".\DATA\netzschleuder"
download_graphs(['spanish_highschools__1'], target_dir)


✘ Failed to download spanish_highschools: 404 Client Error: NOT FOUND for url: https://networks.skewed.de/net/spanish_highschools/files/1.graphml.zst
✘ Failed to decompress .\DATA\netzschleuder/spanish_highschools.gt.zst: [Errno 2] No such file or directory: '.\\DATA\\netzschleuder/spanish_highschools.gt.zst'
